In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder


In [2]:
groceries = []

In [3]:
with open("Groceries.csv","r") as f:groceries = f.read()
groceries = groceries.split("\n")
    

In [5]:
groceries_list = []
for i in groceries:
    groceries_list.append(i.split(","))

In [6]:
te = TransactionEncoder()
te_ary = te.fit(groceries_list).transform(groceries_list)
te_ary

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [20]:
fp_df = pd.DataFrame(te_ary,columns=te.columns_)
one_freq=fp_df.sum().reset_index()
one_freq.columns =['Items',"Freq"]
one_freq.sort_values(by = 'Freq',ascending = False)

,Items,Freq
166,whole milk,2513
103,other vegetables,1903
123,rolls/buns,1809
139,soda,1715
167,yogurt,1372
...,...,...
6,bags,4
80,kitchen utensil,4
115,preservation products,2
5,baby food,1


In [12]:
fp_df.shape

(9835, 169)

In [21]:
#FOR min_support=0.4
# Create frequent itemsets
itemsets=apriori(fp_df,min_support=0.1,use_colnames=True)
# and convert into rules
itemsets.sort_values(by = 'support',ascending = False)

,support,itemsets
6,0.255516,(whole milk)
1,0.193493,(other vegetables)
2,0.183935,(rolls/buns)
4,0.174377,(soda)
7,0.139502,(yogurt)
0,0.110524,(bottled water)
3,0.108998,(root vegetables)
5,0.104931,(tropical fruit)


In [24]:
rules=association_rules(itemsets,metric='confidence',min_threshold=0.1)
rules=rules[['antecedents','consequents','support','confidence','lift']]
rules=rules[rules['lift']>1]
rules.sort_values(by='lift',ascending=False)

,antecedents,consequents,support,confidence,lift


In [25]:
def gen_rules(min_sup,min_coef):
    itemsets=apriori(fp_df,min_support=min_sup,use_colnames=True)
    #and convert into rules
    rules=association_rules(itemsets,metric='confidence',min_threshold=min_coef)
    rules=rules[['antecedents','consequents','support','confidence','lift']]
    rules=rules[rules['lift']>1]
    return rules.sort_values(by='lift',ascending=False)
    

In [26]:
import gradio as gr
demo=gr.Interface(gen_rules,
                 inputs=[gr.Number(value=0.01,step=0.01,minimum=0.0001,maximum=1),
                        gr.Number(value=0.01,step=0.01,minimum=0.0001,maximum=1)],
                 outputs='dataframe')

if __name__=="__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
